# Data Mining Project

## Imports and Datasets

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import seaborn as sns
import math

In [ ]:
account = pd.read_csv('../data/account.csv',delimiter=';')
card_dev = pd.read_csv('../data/card_dev.csv',delimiter=';')
client = pd.read_csv('../data/client.csv',delimiter=';')
disp = pd.read_csv('../data/disp.csv',delimiter=';')
district = pd.read_csv('../data/district.csv',delimiter=';')
loan_dev = pd.read_csv('../data/loan_dev.csv',delimiter=';')
trans_dev = pd.read_csv('../data/trans_dev.csv',delimiter=';', dtype={'bank': str})

## Data Understanding

### Build Dataset

In [ ]:
def parse_date(date):
  year = int(str(date)[0:2])
  month = int(str(date)[2:4])
  day = int(str(date)[4:6])
  return { 'year': year, 'month': month, 'day': day }


def parse_gender(row, birth_date):
  female = birth_date['month'] >= 50

  if female:
    row['gender'] = 'female' 
    birth_date['month'] -= 50
  else:
    row['gender'] = 'male' 


def calculate_age_loan(row):
  date_loan = row['date_loan']
  birth_number = row['birth_number']

  birth_date = parse_date(birth_number)

  parse_gender(row, birth_date)
  
  if (date_loan is None):
    date_loan = parse_date(row['date_loan'])
    date_loan = date_loan['year'] - birth_date['year'] - ((date_loan['month'], date_loan['day']) < (birth_date['month'], birth_date['day']))
  
  row['age_loan'] = date_loan
    
  return row

The mean for the unemployment rate and the total number of offenses committed are calculated in the following two cells of code.

Any missing value is set to the value of the column whose value is present, e.g. if the value for the "unemploymant rate '95" is not present in a certain row, it is set to the "unemploymant rate '96" value.

The reasoning behind this strategy was that, when taken into account over the course of just one year, neither the unemployment rate nor the number of crimes would be significant. 

In [ ]:
def nan_unemploymant_rate(year): return district["unemploymant rate '" + str(year) + " "].isna()

# convert '?' to NaN

district['unemploymant rate \'95 '] = pd.to_numeric(district['unemploymant rate \'95 '], errors='coerce')
district['unemploymant rate \'96 '] = pd.to_numeric(district['unemploymant rate \'96 '], errors='coerce')

# NaN values will be equaled to the value of the other column

district.loc[nan_unemploymant_rate(95), 'unemploymant rate \'95 '] = district['unemploymant rate \'96 ']
district.loc[nan_unemploymant_rate(96), 'unemploymant rate \'96 '] = district['unemploymant rate \'95 ']

# create column with mean from both years and drop previous and now useless columns

district['unemploymant_rate'] = district[['unemploymant rate \'95 ', 'unemploymant rate \'96 ']].mean(axis=1)
district = district.drop(['unemploymant rate \'95 ', 'unemploymant rate \'96 '], axis= 1).reset_index()

In [ ]:
def nan_commited_crimes(year): return district["no. of commited crimes '" + str(year) + " "].isna()

# convert '?' to NaN

district['no. of commited crimes \'95 '] = pd.to_numeric(district['no. of commited crimes \'95 '], errors='coerce')
district['no. of commited crimes \'96 '] = pd.to_numeric(district['no. of commited crimes \'96 '], errors='coerce')

# NaN values will be equaled to the value of the other column

district.loc[nan_commited_crimes(95), 'no. of commited crimes \'95 '] = district['no. of commited crimes \'96 ']
district.loc[nan_commited_crimes(96), 'no. of commited crimes \'96 '] = district['no. of commited crimes \'95 ']

# create column with mean from both years and drop previous and now useless columns

district['commited_crimes'] = district[['no. of commited crimes \'95 ', 'no. of commited crimes \'96 ']].mean(axis=1)
district = district.drop(['no. of commited crimes \'95 ', 'no. of commited crimes \'96 '], axis= 1).reset_index() 

In [ ]:
# join account, loan, disposition and client
df = account.merge(loan_dev, on='account_id', suffixes=('','_loan'), how='right')
df = df.merge(disp, on='account_id', suffixes=('','_disp'), how='left')
df = df.merge(client, on='client_id',suffixes=('','_client'), how='left')

df.drop(columns='district_id', axis=1, inplace=True)

# create age_at_loan and gender column
df = df.apply(lambda row: calculate_age_loan(row), axis=1)

# join demograph
district.rename(columns={'code ':'code'}, inplace=True)
df = df.merge(district, left_on='district_id_client', right_on='code', how='left')

# join creditcard
df = df.merge(card_dev, on='disp_id', suffixes=('', '_card'), how='left')

# join transactions
df_transactions = account.merge(loan_dev, on='account_id', suffixes=('','_loan'), how='right').merge(trans_dev, on='account_id', suffixes=('', '_transaction'), how='left')
df

### Exploratory Analysis

- analise de dados sem graficos
- graficos de dispersao
- bitmaps, coordenadas paralelas
- ver varios atributos ao mm tempo
- matrizes de correlaçao
- graficos de dispersao

In [ ]:
df.info()

In [ ]:
df_transactions.info()

In [ ]:
# correlation analysis

corr = df.corr()
mask = np.triu(corr)

fig = plt.figure(figsize=(25,15))

colormap = sns.diverging_palette(250, 20, as_cmap=True)
sns.heatmap(
    corr, 
    annot=True, 
    fmt='.2f',
    cbar_kws={'shrink': .5},
    mask=mask,
    cmap=colormap).set(title='Correlation Matrix')

plt.show()

In [ ]:
loan_dev['status'].value_counts().plot.bar()
plt.xlabel('Status')
plt.ylabel('Count')
plt.title('Status count for bank loans')

In [ ]:
disp_count = df.groupby(['account_id'])['disp_id'].nunique() # number of disponents of each account

df = df.merge(disp_count, on='account_id', suffixes=('', '_count'), how='left')

In [ ]:
no_dups_df = df.drop_duplicates(subset=['account_id']) #remove duplicates
no_dups_df.groupby('disp_id_count')['status'].value_counts().unstack(1).plot.bar()

plt.xlabel('Disponents')
plt.ylabel('Count by number of disponents')
plt.legend(['Status = -1', 'Status = 1'])
plt.title('Status count by number of account disponents')

In [ ]:
sns.histplot(data=df[df['status'] == 1], x='commited_crimes', color="skyblue", kde=True)
sns.histplot(data=df[df['status'] == -1], x='commited_crimes', color="red", kde=True)

plt.xlabel('Commited crimes')
plt.legend(['Status 1', 'Status -1'])
plt.title('Distribution of commited crimes per status')
plt.show()

In [ ]:
sns.histplot(data=df[df['status'] == 1], x='unemploymant_rate', color="skyblue", kde=True)
sns.histplot(data=df[df['status'] == -1], x='unemploymant_rate', color="red", kde=True)

plt.xlabel('Unemploymant Rate')
plt.legend(['Status 1', 'Status -1'])
plt.title('Distribution of unemploymant rate per status')
plt.show()

In [ ]:
df['diff_salary_loan'] = df['average salary '] - df['payments']

In [ ]:
sns.histplot(data=df[df['status'] == 1], x='diff_salary_loan', color="skyblue", kde=True)
sns.histplot(data=df[df['status'] == -1], x='diff_salary_loan', color="red", kde=True)

plt.xlabel('Salary difference according to loan payment')
plt.legend(['Status 1', 'Status -1'])
plt.title('Distribution of salary loan difference per status')
plt.show()

In [ ]:
df.groupby('gender')['status'].value_counts().unstack(1).plot.bar()

plt.xlabel('Gender')
plt.ylabel('Count')
plt.legend(['Status = -1', 'Status = 1'])
plt.title('Status count by gender')

In [ ]:
transaction_count = df_transactions.groupby(['account_id'])['trans_id'].nunique()

df_transactions = df_transactions.merge(transaction_count, on='account_id', suffixes=['_', '_count'], how='left')

In [ ]:
count_transactions_per_type = df_transactions.groupby(['account_id', 'type']).size().unstack(fill_value=0)
count_transactions_per_type['debit'] = count_transactions_per_type['withdrawal'] + count_transactions_per_type['withdrawal in cash']
count_transactions_per_type.drop(labels=['withdrawal', 'withdrawal in cash'], axis=1, inplace=True)
count_transactions_per_type['credit_debit_ratio'] = count_transactions_per_type['credit'] / count_transactions_per_type['debit']

df_transactions = df_transactions.merge(count_transactions_per_type, on='account_id', how='left')
df_transactions.replace([np.inf, -np.inf], 0, inplace=True)
df_transactions.rename({ 
    'trans_id_count': 'transactions_count', 
    'credit': 'credits_count', 
    'debit': 'debits_count' }, 
    axis=1, inplace=True)


In [ ]:
df_transactions

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15,5))

loan_dev.boxplot(column='amount', by='status', figsize=(5,5), ax=axs[0])
loan_dev.boxplot(column='payments', by='status', figsize=(5,5), ax=axs[1])
loan_dev.boxplot(column='duration', by='status', figsize=(5,5), ax=axs[2])

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15,5))

df.boxplot(column='age_loan', by='status', figsize=(5,5), ax=axs[0])
df.boxplot(column='average salary ', by='status', figsize=(5,5), ax=axs[1])
df.boxplot(column='diff_salary_loan', by='status', figsize=(5,5), ax=axs[2])

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,5))

df_transactions.boxplot(column='transactions_count', by='status', figsize=(5,5), ax=axs[0])
df_transactions.boxplot(column='credit_debit_ratio', by='status', figsize=(5,5), ax=axs[1])

In [ ]:
df.info()

## Data Preparation

- assessment of dimensions of data quality
- (cleaning): redundancy
- (cleaning): missing data
- (cleaning): outliers
- data transformation for compatibility with algorithms
- feature engineering from tabular data
- sampling for domain-specific purposes
- sampling for development
- imbalanced data
- feature selection

In [ ]:
df

##### Redundancy

In [ ]:
# drop duplicates and remove disp_id column (because we already have disp_id_count) 

df.drop_duplicates(subset='account_id', inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.columns

##### Missing Data

In [ ]:
# deal with null values
df.drop(labels=['disp_id', 'card_id', 'type_card', 'issued'], axis=1, inplace=True)

##### Outliers

##### Other data preparation operations

In [ ]:
# drop unnecessary columns
# deal with non numerical values
# standardize columns names
# ... ?